In [1]:
import json
%load_ext autoreload
%autoreload 2

In [2]:
# ! pip uninstall numba -y && pip install numba

In [3]:
import os
import sys

# Получаем абсолютный путь к корневой директории проекта (директория выше текущей)
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Добавляем корневую директорию в sys.path
if root_path not in sys.path:
    sys.path.append(root_path)

In [4]:
from model_loaders import load_ss_model
import weightwatcher as ww
from matplotlib import pyplot as plt
from pipeline import separate_audio
import torch
from utils import parse_yaml
from models.clap_encoder import CLAP_Encoder
import IPython.display as ipd
from models.audiosep_lora_and_tuned_embeddings import AudioSepLoraAndTunedEmbeddings

2024-05-05 00:27:15.085090: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 00:27:15.085116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 00:27:15.085857: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-05 00:27:15.089780: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 00:27:15.614856: W tensorflow/compiler/tf2

In [18]:
SS_CONFIG_PATH = '../config/audiosep_base.yaml'
CLAP_CKPT_PATH = '../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt'
AUDIOSEP_CKPT_PATH = '../checkpoint/audiosep_base_4M_steps.ckpt'
device = torch.device('cuda')
configs = parse_yaml(SS_CONFIG_PATH)

Подготавливаем несколько миксов

In [58]:
from pydub import AudioSegment
import json
import random

def mix_wav_files(files, output_path):
    mixed_sound = AudioSegment.from_file(files[0])

    for file in files[1:]:
        next_sound = AudioSegment.from_file(file)
        mixed_sound = mixed_sound.overlay(next_sound)

    mixed_sound.export(output_path, format='wav')

def select_random_elements(data, n_elements, seed=None):
    if seed is not None:
        random.seed(seed)

    n_elements = min(n_elements, len(data))

    selected_elements = random.sample(data, n_elements)
    return selected_elements

test_files = []
classes = []
mix_cnt = 1
seed = 4
with open('../datafiles/dota2.train.json') as file:
    parsed_json = json.load(file)['data']
    random_mixes = select_random_elements(parsed_json, mix_cnt, seed)
    test_files = [f'../{mix["wav"]}' for mix in random_mixes]
    classes = [mix['caption'] for mix in random_mixes]

mixture_dir = 'dota2_mixtures'
if not os.path.exists(mixture_dir):
    os.makedirs(mixture_dir)

mixture_path = f'{mixture_dir}/{mix_cnt}_{seed}.wav'
mix_wav_files(test_files, mixture_path)

In [59]:
query_encoder = CLAP_Encoder(pretrained_path = CLAP_CKPT_PATH).eval().to(device)
base_model = load_ss_model(configs=configs, checkpoint_path=AUDIOSEP_CKPT_PATH, query_encoder=query_encoder).eval().to(device)

2024-05-05 01:06:36,840 - INFO - Loading HTSAT-base model config.
2024-05-05 01:06:38,192 - INFO - Loading pretrained HTSAT-base-roberta weights (../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt).


In [60]:
checkpoint_path = '../checkpoints/final/dota2/lora_embeddings/final.ckpt'
checkpoint_path = '../checkpoints/train_audiosep_lora_and_tuned_embeddings/audiosep_lora_and_tuned_embeddings_dota2,args=base,timestamp=1714683165.3177924/epoch=49.ckpt'
query_encoder_for_lora = CLAP_Encoder(pretrained_path = CLAP_CKPT_PATH).eval().to(device)
base_model_for_lora = load_ss_model(configs=configs, checkpoint_path=AUDIOSEP_CKPT_PATH, query_encoder=query_encoder_for_lora).eval().to(device)

lora_model = AudioSepLoraAndTunedEmbeddings.load_from_checkpoint(
    checkpoint_path=checkpoint_path,
    strict=False,
    pretrained_audiosep_model = base_model_for_lora,
    loss_function=None,
    waveform_mixer=None,
    lr_lambda_func=None
) \
    .eval() \
    .to(device)

merged_lora_model = lora_model.model.merge_and_unload()
merged_lora_model.query_encoder = lora_model.model.query_encoder

2024-05-05 01:06:41,015 - INFO - Loading HTSAT-base model config.
2024-05-05 01:06:42,365 - INFO - Loading pretrained HTSAT-base-roberta weights (../checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt).


In [62]:
from ipywidgets import widgets
test_files = [mixture_path]

audio_widgets = {cls: {} for cls in classes + ['original']}

for file in test_files:
    filename = file.split(os.sep)[-1]
    audio_widget = widgets.Audio(value=open(file, "rb").read(), format="wav", controls=True, autoplay=False)
    audio_widgets['original'][filename] = audio_widget

    for cls in classes:
        output_dir = '../separation_result/dota2/'
        os.makedirs(os.path.dirname(output_dir), exist_ok=True)
        output_file = os.path.join(output_dir, f'{filename}_{cls}.wav')
        separate_audio(merged_lora_model, file, 'speech', output_file, device, use_chunk=True)
        audio_widget = widgets.Audio(value=open(output_file, "rb").read(), format="wav", controls=True, autoplay=False)
        audio_widgets[cls][filename] = audio_widget

Separating audio from [dota2_mixtures/1_4.wav] with textual query: [speech]
Separated audio written to [../separation_result/dota2/1_4.wav_Huskar.wav]


In [63]:
from utils import plot_separation_result

plot_separation_result(audio_widgets)

HTML(value='<style>.widget-label { font-size: 16px; font-weight: bold; }</style>')

HTML(value='<style>.widget-label { font-size: 16px; font-weight: bold; }</style>')